Векторные представления

In [120]:
from google.colab import drive
drive.mount('/content/gdrive') #, force_remount=True

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import os
os.chdir('gdrive/My Drive/Colab Notebooks')

In [ ]:
!pip install pymorphy2
!pip install pymystem3==0.1.10
!pip install rusenttokenize
!pip install razdel

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
import pandas as pd
from lxml import html
import numpy as np
from tqdm import tqdm
from matplotlib import pyplot as plt
from sklearn.decomposition import TruncatedSVD, NMF, PCA
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_distances
from sklearn.ensemble import RandomForestClassifier
import gensim
import numpy as np
from sklearn.cluster import MiniBatchKMeans
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from collections import Counter,defaultdict
from string import punctuation
from razdel import tokenize as razdel_tokenize
import os
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
%matplotlib inline

morph = MorphAnalyzer()
punct = punctuation+'«»—…“”*№–'
stops = set(stopwords.words('russian'))

def normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0].normal_form for word in words if word and word not in stops]

    return ' '.join(words)

def tokenize(text):
    
    tokens = [token.text for token in list(razdel_tokenize(text))]
    tokens = [token for token in tokens if token.isalnum()]

    return ' '.join(tokens)

## Задача 1

Векторизуйте тексты с помощью Word2vec модели, обученной самостоятельно, и с помощью модели, взятой с rusvectores (например вот этой - http://vectors.nlpl.eu/repository/20/180.zip). Обучите 2 модели по определению перефразирования на получившихся векторах и проверьте, что работает лучше. 
Word2Vec нужно обучить на отдельном корпусе (не на парафразах). Можно взять данные из семинара или любые другие. 
!!!! ВАЖНО: Оценивать модели нужно с помощью кросс-валидации (в семинаре не кросс-валидация)! Метрика - f1.

Загрузим датасет для обучения векторной модели

In [ ]:
data = open('wiki_data.txt').read().splitlines()#[:10000]

data_norm = [normalize(tokenize(text)) for text in data]

In [ ]:
data_norm = [text for text in data_norm if text]

In [ ]:
data_norm[:5]

['новостройка нижегородский область сельский посёлок дивеевский район нижегородский область входить состав сатисский сельсовет посёлок расположить км юг село дивеево 1 км запад город саров право берег река вичкинза правый приток река сатис окружить смешанный лес соединить асфальтовый дорога посёлок цыгановка км грунтовый просёлочный дорога посёлок сатис км название новостройка являться сугубо официальный местный население использовать исключительно альтернативный название хитрый употребляться языковой оборот хитрый ранее использовать название песчаный известковый основать год переселенец соседний село аламасовый нарышкино расположить соответственно 8 14 км запад вознесенский район традиционно посёлок жить рабочий совхоз вперёд центр посёлок сатис возле посёлок расположить карьер активно добывать доломитовый мука бутовый камень настоящий время официально закрытый данные 1978 год посёлок новостройка характеризоваться неперспективный насчитываться 24 хозяйство 43 житель водоснабжение осущ

In [ ]:
len(data_norm)

19987

Обучим на этих текстах модель Word2Vec

In [ ]:
w2v = gensim.models.Word2Vec([text.split() for text in data_norm], size=50, sg=1)

In [ ]:
w2v.most_similar('февраль')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('март', 0.9755054116249084),
 ('май', 0.9746302962303162),
 ('апрель', 0.9742509126663208),
 ('сентябрь', 0.9684893488883972),
 ('ноябрь', 0.9670076966285706),
 ('июль', 0.9617149233818054),
 ('июнь', 0.9601282477378845),
 ('декабрь', 0.9578750729560852),
 ('октябрь', 0.9513184428215027),
 ('август', 0.9496786594390869)]

Загрузим предобученную модель

In [ ]:
import gensim
model = gensim.models.KeyedVectors.load_word2vec_format('180/model.bin', binary=True)

In [ ]:
model.most_similar('февраль_NOUN')

[('март_NOUN', 0.9567033052444458),
 ('ноябрь_NOUN', 0.9523458480834961),
 ('декабрь_NOUN', 0.9509049654006958),
 ('апрель_NOUN', 0.949961245059967),
 ('январь_NOUN', 0.9472224712371826),
 ('октябрь_NOUN', 0.937659740447998),
 ('май_NOUN', 0.911928117275238),
 ('июнь_NOUN', 0.9011650681495667),
 ('сентябрь_NOUN', 0.899361252784729),
 ('июль_NOUN', 0.8842827677726746)]

In [ ]:
mapping = {}

for line in open('ru-rnc.map.txt'):
    ms, ud = line.strip('\n').split()
    mapping[ms] = ud

In [ ]:
from pymystem3 import Mystem
m = Mystem()

In [ ]:
def normalize_mystem(text):
    tokens = []
    norm_words = m.analyze(text)
    for norm_word in norm_words:
        if 'analysis' not in norm_word:
            continue
        if not len(norm_word['analysis']):
            lemma = norm_word['text']
            pos = 'UNKN'
        else:
            lemma = norm_word["analysis"][0]["lex"].lower().strip()
            pos = norm_word["analysis"][0]["gr"].split(',')[0]
            pos = pos.split('=')[0].strip()
        pos = mapping[pos]
        tokens.append(lemma+'_'+pos)

    return tokens

In [ ]:
normalize_mystem('Обучить классификатор парафразов на предобученной модели вам нужно будет дома')

['обучать_VERB',
 'классификатор_NOUN',
 'парафраз_NOUN',
 'на_ADP',
 'предобученный_ADJ',
 'модель_NOUN',
 'вы_PRON',
 'нужно_ADV',
 'быть_VERB',
 'дома_ADV']

In [ ]:
model['обучать_VERB'].shape

(300,)

### Загрузим датасет с парафразами

Коллекция состоит из пар предложения (заголовков статей) и метки класса (-1,0,1). -1 не парафраз, 1 - парафраз, 0 - что-то непонятное.

In [ ]:
corpus_xml = html.fromstring(open('paraphrases.xml', 'rb').read())
texts_1 = []
texts_2 = []
classes = []

for p in corpus_xml.xpath('//paraphrase'):
    texts_1.append(p.xpath('./value[@name="text_1"]/text()')[0])
    texts_2.append(p.xpath('./value[@name="text_2"]/text()')[0])
    classes.append(p.xpath('./value[@name="class"]/text()')[0])
    
data = pd.DataFrame({'text_1':texts_1, 'text_2':texts_2, 'label':classes})

In [ ]:
data['text_1_norm'] = data['text_1'].apply(normalize)
data['text_2_norm'] = data['text_2'].apply(normalize)

In [ ]:
data['text_1_norm_2'] = data['text_1'].apply(normalize_mystem).apply(' '.join)
data['text_2_norm_2'] = data['text_2'].apply(normalize_mystem).apply(' '.join)

In [ ]:
data.head(2)

,text_1,text_2,label,text_1_norm,text_2_norm,text_1_norm_2,text_2_norm_2
0,Полицейским разрешат стрелять на поражение по ...,Полиции могут разрешить стрелять по хулиганам ...,0,полицейский разрешить стрелять поражение гражд...,полиция мочь разрешить стрелять хулиган травма...,полицейский_NOUN разрешать_VERB стрелять_VERB ...,полиция_NOUN мочь_VERB разрешать_VERB стрелять...
1,Право полицейских на проникновение в жилище ре...,Правила внесудебного проникновения полицейских...,0,право полицейский проникновение жилище решить ...,правило внесудебный проникновение полицейский ...,право_ADV полицейский_NOUN на_ADP проникновени...,правило_NOUN внесудебный_ADJ проникновение_NOU...


In [ ]:
data.to_csv('wiki_data_norm.csv')

In [ ]:
data = pd.read_csv('wiki_data_norm.csv')

### Определение перефразирования

In [ ]:
def get_embedding(text, model, dim):
    text = text.split()
    words = Counter(text)
    total = len(text)
    vectors = np.zeros((len(words), dim))
    
    for i, word in enumerate(words):
        try:
            v = model[word]
            vectors[i] = v*(words[word]/total)
        except (KeyError, ValueError):
            continue
    
    if vectors.any():
        vector = np.average(vectors, axis=0)
    else:
        vector = np.zeros((dim))
    
    return vector

### модель обученная на wiki.txt

In [132]:
dim = 50
X_text_1_w2v = np.zeros((len(data['text_1_norm']), dim))
X_text_2_w2v = np.zeros((len(data['text_2_norm']), dim))

for i, text in enumerate(data['text_1_norm'].values):
    X_text_1_w2v[i] = get_embedding(text, w2v, dim)
    
for i, text in enumerate(data['text_2_norm'].values):
    X_text_2_w2v[i] = get_embedding(text, w2v, dim)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':


In [134]:
X_text_w2v = np.concatenate([X_text_1_w2v, X_text_2_w2v], axis=1)

In [135]:
X_text_w2v.shape

(7227, 100)

### предобученная модель

In [ ]:
dim = 300
X_text_1_w2v_2 = np.zeros((len(data['text_1_norm_2']), dim))
X_text_2_w2v_2 = np.zeros((len(data['text_2_norm_2']), dim))

for i, text in enumerate(data['text_1_norm_2'].values):
    X_text_1_w2v_2[i] = get_embedding(text, model, dim)
    
for i, text in enumerate(data['text_2_norm_2'].values):
    X_text_2_w2v_2[i] = get_embedding(text, model, dim)

In [ ]:
X_text_w2v_2 = np.concatenate([X_text_1_w2v_2, X_text_2_w2v_2], axis=1)

In [ ]:
X_text_w2v_2.shape

(7227, 600)

In [ ]:
y = data['label'].values

### обучение классификатора

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
clf = RandomForestClassifier(n_estimators=100, max_depth=7, min_samples_leaf=15,
                             class_weight='balanced')

При обученной векторной модели

In [136]:
scores = cross_val_score(clf, X_text_w2v, y, cv=5, scoring='f1_macro')
scores

array([0.44500725, 0.47000512, 0.46672704, 0.36620472, 0.36700659])

In [137]:
scores = cross_val_score(clf, X_text_w2v, y, cv=5, scoring='f1_micro')
scores

array([0.44329184, 0.48340249, 0.48373702, 0.37923875, 0.39515571])

При предобученной векторной модели


In [ ]:
scores = cross_val_score(clf, X_text_w2v_2, y, cv=5, scoring='f1_macro')
scores

array([0.44382883, 0.47171046, 0.47490029, 0.38985459, 0.36236937])

In [121]:
scores = cross_val_score(clf, X_text_w2v_2, y, cv=5, scoring='f1_micro')
scores

array([0.45643154, 0.50968188, 0.50519031, 0.38961938, 0.37231834])

Результаты предобученной модели получились немного лучше, чем у модели, обученной на wiki.txt

## Задача 2

Преобразуйте тексты в векторы в каждой паре 5 методами  - SVD, NMF, Word2Vec (свой и  русвекторовский), Fastext. У вас должно получиться 5 пар векторов для каждой строчки в датасете. Между векторами каждой пары вычислите косинусную близость (получится 5 чисел для каждой пары). 

Постройте обучающую выборку из этих близостей . Обучите любую модель (Логрег, Рандом форест или что-то ещё) на этой выборке и оцените качество на кросс-валидации (используйте микросреднюю f1-меру).  Попробуйте улучить метрику, изменив параметры в методах векторизации.
!!УТОЧНЕНИЕ: модель нужно обучить сразу на всех 5 близостях, а не по 1 модели на каждой близости!


SVD и NMF применяйте к данным напрямую, а w2w и fastext обучите на отдельном корпусе (как в первой части). 

In [ ]:
from sklearn.metrics.pairwise import cosine_distances

Рассчитаем близости для моделей

1. Обученный Word2Vec

In [142]:
w2v_cos = [cosine_distances([v1], [v2])[0][0] for v1, v2 in zip(X_text_1_w2v, X_text_2_w2v)]

2. Предобученный Word2Vec

In [143]:
w2v_2_cos = [cosine_distances([v1], [v2])[0][0] for v1, v2 in zip(X_text_1_w2v_2, X_text_2_w2v_2)]

3. Обученный FastText

In [144]:
ft = gensim.models.FastText([text.split() for text in data_norm], size=50, sg=1)

In [146]:
dim = 50
X_text_1_ft = np.zeros((len(data['text_1_norm']), dim))
X_text_2_ft = np.zeros((len(data['text_2_norm']), dim))

for i, text in enumerate(data['text_1_norm'].values):
    X_text_1_ft[i] = get_embedding(text, ft, dim)
    
for i, text in enumerate(data['text_2_norm'].values):
    X_text_2_ft[i] = get_embedding(text, ft, dim)

ft_cos = [cosine_distances([v1], [v2])[0][0] for v1, v2 in zip(X_text_1_ft, X_text_2_ft)]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':


4. SVD

In [126]:
tfidf = TfidfVectorizer(min_df=3, max_df=0.4, max_features=1000)
tfidf_texts = tfidf.fit_transform(pd.concat([data['text_1_norm'], data['text_2_norm']]))

In [212]:
svd = TruncatedSVD(50)

svd.fit(tfidf_texts)

X_text_1_svd = svd.transform(tfidf.transform(data['text_1_norm']))
X_text_2_svd = svd.transform(tfidf.transform(data['text_2_norm']))

svd_cos = [cosine_distances([v1], [v2])[0][0] for v1, v2 in zip(X_text_1_svd, X_text_2_svd)]

5. NMF

In [213]:
nmf = NMF(50)

nmf.fit(tfidf_texts)

X_text_1_nmf = nmf.transform(tfidf.transform(data['text_1_norm']))
X_text_2_nmf = nmf.transform(tfidf.transform(data['text_2_norm']))

nmf_cos = [cosine_distances([v1], [v2])[0][0] for v1, v2 in zip(X_text_1_nmf, X_text_2_nmf)]

In [214]:
cos_data = pd.DataFrame()
cos_data['w2v_cos'] = w2v_cos
cos_data['w2v_2_cos'] = w2v_2_cos
cos_data['ft_cos'] = ft_cos
cos_data['svd_cos'] = svd_cos
cos_data['nmf_cos'] = nmf_cos
cos_data['label'] = y

In [215]:
cos_data.head()

,w2v_cos,w2v_2_cos,ft_cos,svd_cos,nmf_cos,label
0,0.092836,0.315568,0.045734,1.094003,0.947187,0
1,0.069705,0.245257,0.080087,0.028260,0.018229,0
2,0.046567,0.173560,0.053419,0.602448,0.503438,0
3,0.204856,0.678593,0.161477,0.351544,0.373685,-1
4,0.087557,0.324991,0.061701,0.080357,0.061438,0


In [216]:
clf = LogisticRegression(solver='liblinear')

X, y = cos_data.drop(['label'], axis=1), cos_data['label']

In [217]:
cross_val_scores = cross_val_score(clf, X, y, scoring='f1_micro')
cross_val_scores

array([0.57053942, 0.58990318, 0.616609  , 0.49065744, 0.49411765])

In [218]:
np.mean(cross_val_scores)

0.5523653366643215

In [219]:
cross_val_scores = cross_val_score(clf, X, y, scoring='f1_macro')
cross_val_scores

array([0.43552911, 0.4553767 , 0.47562098, 0.36483633, 0.37018264])

In [220]:
np.mean(cross_val_scores)

0.42030915278421144

Попробуем улучшить результат, поменяв параметры для svd и nmf

In [223]:
svd = TruncatedSVD(200)

svd.fit(tfidf_texts)

X_text_1_svd = svd.transform(tfidf.transform(data['text_1_norm']))
X_text_2_svd = svd.transform(tfidf.transform(data['text_2_norm']))

svd_cos = [cosine_distances([v1], [v2])[0][0] for v1, v2 in zip(X_text_1_svd, X_text_2_svd)]

In [224]:
nmf = NMF(100)

nmf.fit(tfidf_texts)

X_text_1_nmf = nmf.transform(tfidf.transform(data['text_1_norm']))
X_text_2_nmf = nmf.transform(tfidf.transform(data['text_2_norm']))

nmf_cos = [cosine_distances([v1], [v2])[0][0] for v1, v2 in zip(X_text_1_nmf, X_text_2_nmf)]

In [225]:
cos_data['svd_cos'] = svd_cos
cos_data['nmf_cos'] = nmf_cos

In [226]:
cos_data.head()

,w2v_cos,w2v_2_cos,ft_cos,svd_cos,nmf_cos,label
0,0.092836,0.315568,0.045734,0.569322,0.992004,0
1,0.069705,0.245257,0.080087,0.393141,0.012006,0
2,0.046567,0.173560,0.053419,0.344549,0.232557,0
3,0.204856,0.678593,0.161477,0.422222,0.525985,-1
4,0.087557,0.324991,0.061701,0.153723,0.105068,0


In [227]:
X, y = cos_data.drop(['label'], axis=1), cos_data['label']

In [228]:
cross_val_scores = cross_val_score(clf, X, y, scoring='f1_micro')
cross_val_scores

array([0.5670816 , 0.59197787, 0.6200692 , 0.48927336, 0.49550173])

In [229]:
np.mean(cross_val_scores)

0.5527807530139222

In [231]:
cross_val_scores = cross_val_score(clf, X, y, scoring='f1_macro')
cross_val_scores

array([0.43263559, 0.4675792 , 0.48762723, 0.36898356, 0.37214231])

In [232]:
np.mean(cross_val_scores)

0.42579357916317884

Средние значения стали немного побольше.